<a href="https://colab.research.google.com/github/Shalini-Mirani/ADSLAB/blob/main/ADSEXP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Import libraries***

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.impute import SimpleImputer
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Assume 'outcome' is the target variable in the insurance dataset
df = pd.read_csv("/content/insurance.csv")  # Provide the correct URL for your diabetes dataset
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [ ]:
df.describe([0.01,0.05,0.75,0.90,0.99]).T

,count,mean,std,min,1%,5%,50%,75%,90%,99%,max
PassengerId,418.0,1100.500000,120.810458,892.00,896.170000,912.8500,1100.5000,1204.75,1267.3,1304.830,1309.0000
Survived,418.0,0.363636,0.481622,0.00,0.000000,0.0000,0.0000,1.00,1.0,1.000,1.0000
Pclass,418.0,2.265550,0.841838,1.00,1.000000,1.0000,3.0000,3.00,3.0,3.000,3.0000
Age,332.0,30.272590,14.181209,0.17,0.857900,8.0000,27.0000,39.00,50.0,64.000,76.0000
SibSp,418.0,0.447368,0.896760,0.00,0.000000,0.0000,0.0000,1.00,1.0,4.000,8.0000
Parch,418.0,0.392344,0.981429,0.00,0.000000,0.0000,0.0000,0.00,1.3,4.000,9.0000
Fare,417.0,35.627188,55.907576,0.00,6.446828,7.2292,14.4542,31.50,79.2,262.375,512.3292


In [ ]:
correlation_matrix = df.corr()
survived_correlation = correlation_matrix['Survived'].drop('Survived')
top5att = survived_correlation.abs().sort_values(ascending = False).head(5)
print("Top 5 Attributes with highest correlation with Survived: ")
print(top5att)

Top 5 Attributes with highest correlation with Survived: 
Fare           0.191514
Parch          0.159120
Pclass         0.108615
SibSp          0.099943
PassengerId    0.023245
Name: Survived, dtype: float64


In [ ]:
coltocheck = ['Fare','Parch','Pclass','SibSp','PassengerId']
zero_value_counts = df[coltocheck].eq(0).sum()
print("Number of rows with zero values in each column: ")
print(zero_value_counts)

Number of rows with zero values in each column: 
Fare             2
Parch          324
Pclass           0
SibSp          283
PassengerId      0
dtype: int64


In [ ]:
df[['Fare','Parch','Pclass','SibSp','PassengerId']] = df[['Fare','Parch','Pclass','SibSp','PassengerId']]
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
def missing_percent(df):
    missval = df.isnull().sum()
    misspercent = 100 * df.isnull().sum() / len(df)
    misstable = pd.concat([missval, misspercent], axis=1)
    misscol = misstable.rename(
        columns={0: 'Missing Values', 1: 'Percent of Total values'})
    misscol = misscol[
        misscol.iloc[:, 1] != 0].sort_values(
            'Percent of Total values', ascending=False).round(2)
    print("Your selected dataframe has" + str(df.shape[1]) + " columns.\n" +
          "There are" + str(misscol.shape[0]) +
          " columns that have missing values")
    return misscol

In [ ]:
misscolinfo = missing_percent(df)
misscolinfo

Your selected dataframe has12 columns.
There are3 columns that have missing values


,Missing Values,Percent of Total values
Cabin,327,78.23
Age,86,20.57
Fare,1,0.24


In [ ]:
df['Cabin'] = pd.to_numeric(df['Cabin'], errors='coerce')
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df['Fare'] = pd.to_numeric(df['Fare'], errors='coerce')

# Check the type of df['Cabin']
print(type(df['Cabin']))

# Check the type of df['Age']
print(type(df['Age']))

# Check the type of df['Fare']
print(type(df['Fare']))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [ ]:
# coltocheck = ['Cabin','Age','Fare']


def print_outliercount(column):
  Q1 = df[column].quantile(0.25)
  Q3 = df[column].quantile(0.75)
  IQR = Q3-Q1
  lowerbound=Q1-1.5*IQR
  upperbound=Q3+1.5*IQR
  outliercount = df[(df[column]<lowerbound)|(df[column]>upperbound)].shape[0]
  print(f"Outliers count in {column}:{outliercount}")
  df.drop(df[(df[column]<lowerbound)|(df[column]>upperbound)].index,inplace=True)

for column in coltocheck:
  print_outliercount(column)

Outliers count in Cabin:0
Outliers count in Age:2
Outliers count in Fare:53


In [ ]:
df_drop=df.copy()

In [ ]:
drop_cols = misscolinfo[misscolinfo['Percent of Total values']>25]
drop_cols

,Missing Values,Percent of Total values
Cabin,327,78.23


In [ ]:
colnames = drop_cols.index.tolist()
print(colnames)

['Cabin']


In [ ]:
# Ensure 'Cabin' column is present in drop_cols DataFrame
drop_cols = drop_cols[drop_cols.index.isin(['Cabin'])]

# Remove 'Cabin' from colnames list
colnames.remove('Cabin')

# Drop specified columns from df_drop DataFrame
df_drop.drop(colnames, axis=1, inplace=True)
df_drop.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S


In [ ]:
df_drop_rows=df.copy()

In [ ]:
df_drop_rows=df_drop_rows.dropna(subset=colnames)
df_drop_rows.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
# Separate features and target variable
X = titanic_data.drop('charges', axis=1)
y = titanic_data['charges']

In [ ]:
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
print(numerical_features)
categorical_features = X.select_dtypes(include=['object']).columns
print(categorical_features)

Index(['age', 'bmi', 'children'], dtype='object')
Index(['sex', 'smoker', 'region'], dtype='object')


In [ ]:
# Create transformers for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

In [ ]:
# Combine transformers into a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

In [ ]:
# Fit and transform the preprocessor on the dataset
X_transformed = preprocessor.fit_transform(X)

# Display the transformed dataset
transformed_columns = numerical_features.tolist() + categorical_features.tolist()
print("\nTransformed dataset:")
print(pd.DataFrame(X_transformed, columns=transformed_columns))


Transformed dataset:
       age     bmi children     sex smoker     region
0     19.0    27.9      0.0  female    yes  southwest
1     18.0   33.77      1.0    male     no  southeast
2     28.0    33.0      3.0    male     no  southeast
3     33.0  22.705      0.0    male     no  northwest
4     32.0   28.88      0.0    male     no  northwest
...    ...     ...      ...     ...    ...        ...
1333  50.0   30.97      3.0    male     no  northwest
1334  18.0   31.92      0.0  female     no  northeast
1335  18.0   36.85      0.0  female     no  southeast
1336  21.0    25.8      0.0  female     no  southwest
1337  61.0   29.07      0.0  female    yes  northwest

[1338 rows x 6 columns]


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# Assume 'outcome' is the target variable in the insurance dataset
df = pd.read_csv("/content/diabetes.csv")  # Provide the correct URL for your diabetes dataset
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [ ]:
column_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
titanic_data = df
df.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
titanic_data.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [ ]:
titanic_data.dropna()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [ ]:
# Separate features and target variable
X = titanic_data.drop('Outcome', axis=1)
y = titanic_data['Outcome']

In [ ]:
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
print(numerical_features)
categorical_features = X.select_dtypes(include=['object']).columns
print(categorical_features)

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'],
      dtype='object')
Index([], dtype='object')


In [ ]:
# Create transformers for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

In [ ]:
# Combine transformers into a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

In [ ]:
# Fit and transform the preprocessor on the dataset
X_transformed = preprocessor.fit_transform(X)

# Display the transformed dataset
transformed_columns = numerical_features.tolist() + categorical_features.tolist()
print("\nTransformed dataset:")
print(pd.DataFrame(X_transformed, columns=transformed_columns))


Transformed dataset:
     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6.0    148.0           72.0           35.0      0.0  33.6   
1            1.0     85.0           66.0           29.0      0.0  26.6   
2            8.0    183.0           64.0            0.0      0.0  23.3   
3            1.0     89.0           66.0           23.0     94.0  28.1   
4            0.0    137.0           40.0           35.0    168.0  43.1   
..           ...      ...            ...            ...      ...   ...   
763         10.0    101.0           76.0           48.0    180.0  32.9   
764          2.0    122.0           70.0           27.0      0.0  36.8   
765          5.0    121.0           72.0           23.0    112.0  26.2   
766          1.0    126.0           60.0            0.0      0.0  30.1   
767          1.0     93.0           70.0           31.0      0.0  30.4   

     DiabetesPedigreeFunction   Age  
0                       0.627  50.0  
1            

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# Assume 'outcome' is the target variable in the insurance dataset
df = pd.read_csv("/content/tested.csv")  # Provide the correct URL for your diabetes dataset
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
column_names = ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
titanic_data = df
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
titanic_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [ ]:
titanic_data.dropna()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
12,904,1,1,"Snyder, Mrs. John Pillsbury (Nelle Stevenson)",female,23.0,1,0,21228,82.2667,B45,S
14,906,1,1,"Chaffee, Mrs. Herbert Fuller (Carrie Constance...",female,47.0,1,0,W.E.P. 5734,61.1750,E31,S
24,916,1,1,"Ryerson, Mrs. Arthur Larned (Emily Maria Borie)",female,48.0,1,3,PC 17608,262.3750,B57 B59 B63 B66,C
26,918,1,1,"Ostby, Miss. Helene Ragnhild",female,22.0,0,1,113509,61.9792,B36,C
28,920,0,1,"Brady, Mr. John Bertram",male,41.0,0,0,113054,30.5000,A21,S
...,...,...,...,...,...,...,...,...,...,...,...,...
404,1296,0,1,"Frauenthal, Mr. Isaac Gerald",male,43.0,1,0,17765,27.7208,D40,C
405,1297,0,2,"Nourney, Mr. Alfred (Baron von Drachstedt"")""",male,20.0,0,0,SC/PARIS 2166,13.8625,D38,C
407,1299,0,1,"Widener, Mr. George Dunton",male,50.0,1,1,113503,211.5000,C80,C
411,1303,1,1,"Minahan, Mrs. William Edward (Lillian E Thorpe)",female,37.0,1,0,19928,90.0000,C78,Q


In [ ]:
# Separate features and target variable
X = titanic_data.drop('Survived', axis=1)
y = titanic_data['Survived']

In [ ]:
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
print(numerical_features)
categorical_features = X.select_dtypes(include=['object']).columns
print(categorical_features)

Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')
Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')


In [ ]:
# Create transformers for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

In [ ]:
# Combine transformers into a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

In [ ]:
# Fit and transform the preprocessor on the dataset
X_transformed = preprocessor.fit_transform(X)

# Display the transformed dataset
transformed_columns = numerical_features.tolist() + categorical_features.tolist()
print("\nTransformed dataset:")
print(pd.DataFrame(X_transformed, columns=transformed_columns))


Transformed dataset:
    PassengerId Pclass       Age SibSp Parch     Fare  \
0         892.0    3.0      34.5   0.0   0.0   7.8292   
1         893.0    3.0      47.0   1.0   0.0      7.0   
2         894.0    2.0      62.0   0.0   0.0   9.6875   
3         895.0    3.0      27.0   0.0   0.0   8.6625   
4         896.0    3.0      22.0   1.0   1.0  12.2875   
..          ...    ...       ...   ...   ...      ...   
413      1305.0    3.0  30.27259   0.0   0.0     8.05   
414      1306.0    1.0      39.0   0.0   0.0    108.9   
415      1307.0    3.0      38.5   0.0   0.0     7.25   
416      1308.0    3.0  30.27259   0.0   0.0     8.05   
417      1309.0    3.0  30.27259   1.0   1.0  22.3583   

                                             Name     Sex              Ticket  \
0                                Kelly, Mr. James    male              330911   
1                Wilkes, Mrs. James (Ellen Needs)  female              363272   
2                       Myles, Mr. Thomas Francis 